In [1]:
import os
import pyprind
import pickle
import pandas as pd

In [2]:
pbar = pyprind.ProgBar(50000)
labels = {'pos':1, 'neg':0}
df = pd.DataFrame()
for s in ('test','train'):
    for l in ('pos','neg'):
        path = 'C:\\Users\\sam32564\\Downloads\\aclImdb\\%s\\%s' % (s,l)
        for file in os.listdir(path):
            with open(os.path.join(path, file), 'r',encoding="utf-8") as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index = True)
            pbar.update()
df.columns = ['review', 'sentiment']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:02:04


In [3]:
import numpy as np
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('./movie_data.csv', index = False)

In [4]:
df = pd.read_csv('./movie_data.csv')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [5]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array(['The sun is shining',
                'The weather is sweet',
                'The sun is shining and the weather is sweet'])
bag = count.fit_transform(docs)

In [6]:
print(count.vocabulary_)

{'the': 5, 'sun': 3, 'is': 1, 'shining': 2, 'weather': 6, 'sweet': 4, 'and': 0}


In [7]:
print(bag.toarray())

[[0 1 1 1 0 1 0]
 [0 1 0 0 1 1 1]
 [1 2 1 1 1 2 1]]


In [8]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.56 0.56 0.   0.43 0.  ]
 [0.   0.43 0.   0.   0.56 0.43 0.56]
 [0.4  0.48 0.31 0.31 0.31 0.48 0.31]]


In [9]:
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

In [10]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*','',text)
    emotions = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text)
    text = re.sub('[\W]+',' ', text.lower())
    return text

In [11]:
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

In [12]:
preprocessor("</a>This :) is :( a test :-)!")

' this is a test '

In [13]:
df['review'] = df['review'].apply(preprocessor)

In [14]:
def tokenizer(text):
    return text.split()
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [15]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [16]:
#下載停用字
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sam32564\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:] if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

In [18]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(strip_accents=None,
                       lowercase=False,
                       preprocessor=None)
param_grid = [{'vect__ngram_range':[(1,1)],
              'vect__stop_words':[stop, None],
              'vect__tokenizer':[tokenizer,tokenizer_porter],
              'clf__penalty':['l1','l2'],
              'clf__C':[1.0,10.0,100.0]},
             {'vect__ngram_range':[(1,1)],
              'vect__stop_words':[stop, None],
              'vect__tokenizer':[tokenizer,tokenizer_porter],
              'vect__use_idf':[False],
              'vect__norm':[None],
              'clf__penalty':['l1','l2'],
              'clf__C':[1.0,10.0,100.0]}
             ]
lr_tfidf = Pipeline([('vect',tfidf),
                    ('clf',LogisticRegression(random_state=0))])
gs_lr_tfidf = GridSearchCV(lr_tfidf,
                           param_grid,
                          scoring='accuracy',
                          cv=5,
                          verbose=1,
                          n_jobs=-1)
gs_lr_tfidf.fit(X_train,y_train)
             

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 188 tasks      | elapsed: 43.2min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed: 57.6min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=False,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        n

In [20]:
print('Best parameter set: %s' % gs_lr_tfidf.best_params_)

Best parameter set: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x000002CE14E5B3A8>}


In [21]:
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)


CV Accuracy: 0.897


In [22]:
clf = gs_lr_tfidf.best_estimator_


In [23]:
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

Test Accuracy: 0.898


In [24]:
import numpy as np
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ',text.lower()) \
            + ' '.join(emoticons).replace('-','')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

In [25]:
def stream_docs(path):
    with open(path, 'r', encoding="utf-8") as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3],int(line[-2])
            yield text, label

In [26]:
next(stream_docs(path='./movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [27]:
def get_minibatch(doc_stream, size):
    docs, y = [],[]
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [28]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
vect = HashingVectorizer(decode_error='ignore',
                        n_features=2**21,
                        preprocessor=None,
                        tokenizer=tokenizer)
clf = SGDClassifier(loss='log',random_state=1)
doc_stream = stream_docs(path='./movie_data.csv')

In [29]:
import pyprind
pbar = pyprind.ProgBar(45)
classes = np.array([0,1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train,y_train,classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:44


In [30]:
X_test, y_test = get_minibatch(doc_stream,size=5000)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test,y_test))

Accuracy: 0.867


In [31]:
clf = clf.partial_fit(X_test,y_test)

In [32]:
import pickle
import os 
dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)
pickle.dump(stop,open(os.path.join(dest,'stopwords.pkl'),'wb'),protocol=4)
pickle.dump(clf,open(os.path.join(dest,'classifier.pkl'),'wb'),protocol=4)

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer
import os 
import re
import pickle

cur_dir = os.path.dirname(__file__)
stop = pickle.load(open(os.path.join(cur_dir,'stopwords.pkl'), 'rb'))

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub(r'[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)

In [ ]:
import pickle
import re
import os
from vectorizer import vect
cur_dir = os.path.dirname(__file__)
clf = pickle.load(open(os.path.join(cur_dir,'pkl_objects/classifier.pkl'),'rb'))

In [ ]:
import numpy as np
label = {0:'negative',1:'positive'}
example = ['I love this movie']
X = vect.transform(example)
print('Prediction: %s\nProbability: %.2f%%' %\
     (label[clf.predict(X)[0]],
     np.max(clf.predict_proba(X))*100))

In [ ]:
import sqlite3
import os
conn = sqlite3.connect('reviews.sqlite')
c = conn.cursor()
c.execute('CREATE TABLE review_db'
         ' (review TEXT, sentiment INTEGER, date TEXT)')
example1 = 'I love this movie'
c.execute('INSERT INTO review_db'
         '(review, sentiment, date) VALUES'
         '(?,?,DATETIME("NOW"))',(example1,1))
example2 = 'I dislike this movie'
c.execute('INSERT INTO review_db'
         '(review, sentiment, date) VALUES' 
         '(?,?,DATETIME("NOW"))',(example2,0))
conn.commit()
conn.close()

In [2]:
conn = sqlite3.connect('reviews.sqlite')
c = conn.cursor()
c.execute('SELECT * FROM review_db WHERE date BETWEEN "2015-01-01 00:00:00" AND DATETIME("now")')
results= c.fetchall()
conn.close()
print(results)

[('I love this movie', 1, '2020-09-12 08:56:18'), ('I dislike this movie', 0, '2020-09-12 08:56:18')]


In [ ]:
data = pd.read_pickle('classifier.pkl')

In [ ]:
cur_dir = 'C:\\training\\predict_review_app'
clf = pickle.load(open(os.path.join(cur_dir,'pkl_objects/classifier.pkl'),'rb'))

In [ ]:
##更新電影評論分類器
import pickle
import sqlite3
import numpy as np
import os

 #import HashingVectorizer from local dir
from vectorizer import vect

def update_model(db_path, model, batch_size=10000):

    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    c.execute('select * from review_db')

    results = c.fetchmany(batch_size)
    while results:
        data = np.array(results)
        X = data[:, 0]
        y = data[:, 1].astype(int)

        classes = np.array([0,1])
        X_train = vect.transform(X)
        clf.partial_fit(X_train, y, classes=classes)
        results = c.fetchmany(batch_size)

    conn.close()
    return None

cur_dir = os.path.dirname(__file__)

clf = pickle.load(open(os.path.join(cur_dir, 'classifier.pkl'),'rb'))

db = os.path.join(cur_dir, 'review.sqlite')

update_model(db_path = db, model = clf, batch_size = 10000)